In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [52]:
import numpy as np

class Router:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.neighbors = [] 
        self.edges = []  

class Edge:
    def __init__(self, start, end, length):
        self.start = start
        self.end = end
        self.length = length
        self.load = 0  



# sample dataset 

In [53]:


def generate_sample_data(m, n, num_samples):
    data = {
        'Source': [],
        'Destination': [],
        'Current_Router': [],
        'Action': [],
        'Next_Router': []
    }

    for _ in range(num_samples):
        source = (random.randint(0, m-1), random.randint(0, n-1))
        destination = (random.randint(0, m-1), random.randint(0, n-1))
        current_router = source
        action = random.choice(['down', 'up', 'left', 'right'])

        if action == 'down' and current_router[0] < destination[0]:
            next_router = (current_router[0] + 1, current_router[1])
        elif action == 'up' and current_router[0] > destination[0]:
            next_router = (current_router[0] - 1, current_router[1])
        elif action == 'right' and current_router[1] < destination[1]:
            next_router = (current_router[0], current_router[1] + 1)
        elif action == 'left' and current_router[1] > destination[1]:
            next_router = (current_router[0], current_router[1] - 1)
        else:
            next_router = current_router  

        data['Source'].append(source)
        data['Destination'].append(destination)
        data['Current_Router'].append(current_router)
        data['Action'].append(action)
        data['Next_Router'].append(next_router)

    return pd.DataFrame(data)

def observation(df):
    observations_data = {
        'Source_X': [],
        'Source_Y': [],
        'Destination_X': [],
        'Destination_Y': [],
        'Current_X': [],
        'Current_Y': [],
        'Action':[],
        'Next_X': [],
        'Next_Y': []
    }

    for index, row in df.iterrows():
        source_x, source_y = row['Source']
        dest_x, dest_y = row['Destination']
        current_x, current_y = row['Current_Router']
        action=row['Action']
        next_x, next_y = row['Next_Router'] if row['Action'] == 'send' else (current_x, current_y)

        observations_data['Source_X'].append(source_x)
        observations_data['Source_Y'].append(source_y)
        observations_data['Destination_X'].append(dest_x)
        observations_data['Destination_Y'].append(dest_y)
        observations_data['Current_X'].append(current_x)
        observations_data['Current_Y'].append(current_y)
        observations_data['Action'].append(action)
        observations_data['Next_X'].append(next_x)
        observations_data['Next_Y'].append(next_y)

    return pd.DataFrame(observations_data)


m, n = 5, 5  
num_samples = 20

sample_data = generate_sample_data(m, n, num_samples)

observations_df = observation(sample_data)

action_mapping = {'up': 0, 'right': 1, 'down': 2, 'left': 3}
observations_df['Action'] = observations_df['Action'].map(action_mapping)

print(observations_df)


    Source_X  Source_Y  Destination_X  Destination_Y  Current_X  Current_Y  \
0          2         4              3              0          2          4   
1          0         1              1              4          0          1   
2          2         2              4              4          2          2   
3          4         0              3              4          4          0   
4          4         0              4              0          4          0   
5          0         4              4              4          0          4   
6          4         3              3              1          4          3   
7          0         4              4              4          0          4   
8          0         0              0              4          0          0   
9          0         2              2              2          0          2   
10         1         4              3              3          1          4   
11         4         0              4              3          4 

In [54]:
sample_data.head()

,Source,Destination,Current_Router,Action,Next_Router
0,"(2, 4)","(3, 0)","(2, 4)",right,"(2, 4)"
1,"(0, 1)","(1, 4)","(0, 1)",right,"(0, 2)"
2,"(2, 2)","(4, 4)","(2, 2)",up,"(2, 2)"
3,"(4, 0)","(3, 4)","(4, 0)",up,"(3, 0)"
4,"(4, 0)","(4, 0)","(4, 0)",right,"(4, 0)"


# Defining Gan

In [55]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            nn.Linear(256, output_dim),
            nn.Tanh()  # Assuming your data is normalized
        )
        
    def forward(self, x):
        return self.net(x)


class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.net(x)


## Making a dataloader for the gan from our dataset


In [56]:

observations_np = observations_df.to_numpy()
observations_tensor = torch.tensor(observations_np, dtype=torch.float64)

from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(observations_tensor)

batch_size = 64  
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)



In [61]:
# Define the size of the input noise vector
noise_vector_size = 100  # Example size, adjust according to your generator's input layer

# Generate noise based on the specified size
noise = torch.randn(batch_size, noise_vector_size, device=device)


In [67]:
def train_gan(generator, discriminator, gen_optimizer, disc_optimizer, real_data_loader, epochs, device):
    criterion = nn.BCELoss()
    
    for epoch in range(epochs):
        for real_data in real_data_loader:
            real_data = real_data[0].to(device)  # Assuming real_data is the first element
            real_data = real_data.float()  # Convert real_data to float to match the discriminator's weight data type
            batch_size = real_data.size(0)

            # Zero the discriminator's gradient
            disc_optimizer.zero_grad()

            # Generate real labels
            real_labels = torch.ones(batch_size, 1, device=device)

            # Forward pass real data through discriminator
            real_output = discriminator(real_data)
            real_loss = criterion(real_output, real_labels)

            # Generate fake data and process through discriminator
            noise = torch.randn(batch_size, noise_vector_size, device=device)  # Use the adjusted line
            fake_data = generator(noise)
            fake_labels = torch.zeros(batch_size, 1, device=device)
            fake_output = discriminator(fake_data.detach())
            fake_loss = criterion(fake_output, fake_labels)

            # Update discriminator
            disc_loss = real_loss + fake_loss
            disc_loss.backward()
            disc_optimizer.step()

            # Update generator
            gen_optimizer.zero_grad()
            trick_labels = torch.ones(batch_size, 1, device=device)
            trick_output = discriminator(fake_data)
            gen_loss = criterion(trick_output, trick_labels)
            gen_loss.backward()
            gen_optimizer.step()

        if epoch % 10 == 0:  # Adjust logging frequency as needed
            print(f'Epoch [{epoch+1}/{epochs}], Loss D: {disc_loss.item()}, Loss G: {gen_loss.item()}')


In [63]:
import torch
import torch.optim as optim

gen_input_dim = 100  
gen_output_dim = len(observations_tensor[0])  
disc_input_dim = gen_output_dim  


generator = Generator(input_dim=gen_input_dim, output_dim=gen_output_dim)
discriminator = Discriminator(input_dim=disc_input_dim)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)


gen_optimizer = optim.Adam(generator.parameters(), lr=0.002)
disc_optimizer = optim.Adam(discriminator.parameters(), lr=0.002)


In [68]:
epochs = 200  # Number of epochs for training

train_gan(
    generator=generator, 
    discriminator=discriminator, 
    gen_optimizer=gen_optimizer, 
    disc_optimizer=disc_optimizer, 
    real_data_loader=data_loader, 
    epochs=epochs, 
    device=device
)


Epoch [1/200], Loss D: 1.3027633428573608, Loss G: 0.6584224700927734
Epoch [11/200], Loss D: 0.9827607870101929, Loss G: 0.9108744859695435
Epoch [21/200], Loss D: 0.5183703899383545, Loss G: 1.3145405054092407
Epoch [31/200], Loss D: 0.10974805057048798, Loss G: 3.077634334564209
Epoch [41/200], Loss D: 0.18329884111881256, Loss G: 4.525663375854492
Epoch [51/200], Loss D: 0.02407725900411606, Loss G: 4.492980480194092
Epoch [61/200], Loss D: 0.0038469904102385044, Loss G: 6.033292293548584
Epoch [71/200], Loss D: 0.00025343368179164827, Loss G: 8.849454879760742
Epoch [81/200], Loss D: 0.00020972291531506926, Loss G: 9.49461555480957
Epoch [91/200], Loss D: 0.0221200343221426, Loss G: 9.249526977539062
Epoch [101/200], Loss D: 0.0023076217621564865, Loss G: 8.721237182617188
Epoch [111/200], Loss D: 0.0004910351126454771, Loss G: 10.333526611328125
Epoch [121/200], Loss D: 0.00020536608644761145, Loss G: 10.64317512512207
Epoch [131/200], Loss D: 0.11869251728057861, Loss G: 8.74839

## Viewing the gan's synthetic data

In [77]:

noise_vector_size = 100
batch_size = 10  

noise = torch.randn(batch_size, noise_vector_size, device=device)


with torch.no_grad():
    generated_data = generator(noise)
    
df_generated = pd.DataFrame(generated_data.cpu().numpy())
print(df_generated.head(10))  


          0         1         2         3         4         5         6  \
0  0.964730 -0.985407  0.988251  0.998960 -0.882355 -0.978525  0.871041   
1  0.987782 -0.975466  0.996412  0.999611  0.382179 -0.993216  0.993199   
2  1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000  1.000000   
3  0.974723 -0.969872  0.988741  0.998967  0.450532 -0.987203  0.976958   
4  1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000  1.000000   
5  0.992475 -0.991197  0.995878  0.999962 -0.954226 -0.992383  0.963122   
6  0.983048 -0.995304  0.996103  0.999703 -0.945527 -0.992402  0.961114   
7  0.986123 -0.993054  0.996254  0.999778 -0.978585 -0.995627  0.950006   
8  0.993814 -0.991044  0.993498  0.999896 -0.565986 -0.992772  0.954848   
9  1.000000 -1.000000  1.000000  1.000000  1.000000 -1.000000  1.000000   

          7         8  
0 -0.612034  0.987279  
1 -0.958979  0.997326  
2 -1.000000  1.000000  
3 -0.903850  0.997127  
4 -1.000000  1.000000  
5 -0.804992  0.994416  
6 -0.6

In [76]:
import pandas as pd
import numpy as np
import torch

# Decoding function
def decode_gan_output(gan_output, mesh_size_x, mesh_size_y):
    decoded_data = []
    for row in gan_output:
        source_x = int((row[0] + 1) * mesh_size_x / 2)
        source_y = int((row[1] + 1) * mesh_size_y / 2)
        destination_x = int((row[2] + 1) * mesh_size_x / 2)
        destination_y = int((row[3] + 1) * mesh_size_y / 2)
        current_x = int((row[4] + 1) * mesh_size_x / 2)
        current_y = int((row[5] + 1) * mesh_size_y / 2)
        # Assuming action is discretized into 4 steps
        action_idx = np.argmax(row[6:10])  # This part needs adjustment based on your GAN's design
        action = ['up', 'down', 'left', 'right'][action_idx]  # Example mapping
        next_x = int((row[7] + 1) * mesh_size_x / 2)
        next_y = int((row[8] + 1) * mesh_size_y / 2)
        decoded_data.append([source_x, source_y, destination_x, destination_y, current_x, current_y, action, next_x, next_y])
    return pd.DataFrame(decoded_data, columns=['Source_X', 'Source_Y', 'Destination_X', 'Destination_Y', 'Current_X', 'Current_Y', 'Action', 'Next_X', 'Next_Y'])


df_generated_values = df_generated.to_numpy()


decoded_df = decode_gan_output(df_generated_values, 5, 5)
print(decoded_df)


   Source_X  Source_Y  Destination_X  Destination_Y  Current_X  Current_Y  \
0         4         0              4              4          0          0   
1         4         0              4              4          0          0   
2         4         0              4              4          0          0   
3         4         0              4              4          1          0   
4         4         0              4              4          0          0   
5         4         0              4              4          0          0   
6         5         0              5              5          5          0   
7         5         0              5              5          5          0   
8         4         0              4              4          0          0   
9         4         0              4              4          0          0   

  Action  Next_X  Next_Y  
0   left       1       4  
1   left       0       4  
2   left       1       4  
3   left       0       4  
4   left       0 